<a href="https://colab.research.google.com/github/FrenitaFernandes/eye_disease_project/blob/main/Copy_of_Eye_disease_detection_convnext_tiny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import All Necessary Libraries
# ======================================================================================
import os
import zipfile
import shutil
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchsummary import summary
from google.colab import drive
import timm


In [ ]:
# ======================================================================================
# Step 2: Mount Drive and Extract Dataset
# ======================================================================================
# Mount your Google Drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
zip_path = '/content/drive/MyDrive/Frenita (1)/eyeimage.zip'
extract_path = '/content/eye_dataset'

# Clean up previous extractions if they exist
if os.path.exists(extract_path):
    shutil.rmtree(extract_path)
os.makedirs(extract_path, exist_ok=True)

print(f"Extracting {zip_path} to {extract_path}...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete!")


Extracting /content/drive/MyDrive/Frenita (1)/eyeimage.zip to /content/eye_dataset...
Extraction complete!


In [ ]:
# ======================================================================================
# Step 3: Reorganize Files and Verify
# ======================================================================================
print("\nReorganizing extracted files...")
source_folder = os.path.join(extract_path, 'eyeimage')

# Move class folders (cataract, normal, etc.) up one level
if os.path.isdir(source_folder):
    for item_name in os.listdir(source_folder):
        shutil.move(os.path.join(source_folder, item_name), extract_path)
    os.rmdir(source_folder)
    print(" Files moved to the correct directory.")
else:
    print(" Warning: 'eyeimage' folder not found. Assuming files are already correctly placed.")

data_dir = extract_path
print(f"Dataset path has been set to: {data_dir}")

# Verify that the class folders are now directly inside extract_path
print("\nVerifying extracted class folders...")
try:
    class_folders = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    if class_folders:
        print(f" Success! Found {len(class_folders)} class folders:")
        for folder_name in sorted(class_folders):
            print(f"  - {folder_name}")
    else:
        print(" Warning: No class folders were found at the specified path.")
except FileNotFoundError:
    print(f" Error: The directory '{data_dir}' was not found.")



Reorganizing extracted files...
 Files moved to the correct directory.
Dataset path has been set to: /content/eye_dataset

Verifying extracted class folders...
 Success! Found 4 class folders:
  - cataract
  - diabetic_retinopathy
  - glaucoma
  - normal


In [ ]:
# Step 4: Install Required Libraries
# ======================================================================================
!pip install -q timm albumentations



In [ ]:
# Step 5: Intelligent Cropping and Custom PyTorch Dataset
# ======================================================================================
def crop_image_from_gray(img):
    if img is None: return None
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
    _, thresh = cv2.threshold(gray_img, 45, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return img
    c = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    return img[y:y+h, x:x+w]

class EyeDiseaseDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        if self.transform:
            image = self.transform(image=image)['image']
        label = self.labels[idx]
        return image, torch.tensor(label, dtype=torch.long)


In [ ]:
# Step 6: Prepare Data and Define Augmentations
# ======================================================================================
image_paths = glob.glob(os.path.join(data_dir, '*', '*.png')) + glob.glob(os.path.join(data_dir, '*', '*.jpg'))
class_names = sorted([d.name for d in os.scandir(data_dir) if d.is_dir()])
class_to_idx = {name: i for i, name in enumerate(class_names)}
labels = [class_to_idx[os.path.basename(os.path.dirname(p))] for p in image_paths]

print(f"\nFound {len(image_paths)} images belonging to {len(class_names)} classes.")

train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, random_state=42, stratify=labels
)

IMG_SIZE = 224
train_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
val_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

train_dataset = EyeDiseaseDataset(train_paths, train_labels, transform=train_transform)
val_dataset = EyeDiseaseDataset(val_paths, val_labels, transform=val_transform)

BATCH_SIZE = 32
# --- CPU Change: pin_memory is set to False ---
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=False)



Found 3119 images belonging to 4 classes.


In [ ]:
# ======================================================================================
# Step 7: Define the ConvNeXt Model
# ======================================================================================
def create_model(num_classes=4, pretrained=True):
    model = timm.create_model('convnext_tiny', pretrained=pretrained)
    n_features = model.head.fc.in_features
    model.head.fc = nn.Linear(n_features, num_classes)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = create_model(num_classes=len(class_names)).to(device)

print(f"\nModel will be trained on: {device}")
print("Model Summary:")
summary(model, (3, IMG_SIZE, IMG_SIZE))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]


Model will be trained on: cpu
Model Summary:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 56, 56]           4,704
       LayerNorm2d-2           [-1, 96, 56, 56]             192
          Identity-3           [-1, 96, 56, 56]               0
            Conv2d-4           [-1, 96, 56, 56]           4,800
         LayerNorm-5           [-1, 56, 56, 96]             192
            Linear-6          [-1, 56, 56, 384]          37,248
              GELU-7          [-1, 56, 56, 384]               0
           Dropout-8          [-1, 56, 56, 384]               0
          Identity-9          [-1, 56, 56, 384]               0
           Linear-10           [-1, 56, 56, 96]          36,960
          Dropout-11           [-1, 56, 56, 96]               0
              Mlp-12           [-1, 56, 56, 96]               0
         Identity-13           [-1, 96, 56, 56]          

In [ ]:
# ======================================================================================
# Step 8: Training and Validation Loops (CPU Version)
# ======================================================================================
EPOCHS = 15
LEARNING_RATE = 1e-4

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)
# The GradScaler for mixed-precision is removed for CPU training.

def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    progress_bar = tqdm(dataloader, desc="Training", colour="green")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
        progress_bar.set_postfix(loss=loss.item())

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    progress_bar = tqdm(dataloader, desc="Validation", colour="red")

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_predictions += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_predictions.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()


In [ ]:
# Step 9: Main Training Execution (CPU Version)
# ======================================================================================
best_val_accuracy = 0.0

print("\nStarting model training on CPU... (This will be slow 🐢)")
for epoch in range(EPOCHS):
    print(f"--- Epoch {epoch+1}/{EPOCHS} ---")

    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_one_epoch(model, val_loader, criterion, device)

    scheduler.step()

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}\n")

    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), 'best_model_cpu.pth')
        print(f"New best model saved with accuracy: {best_val_accuracy:.4f}")

print("Training finished!")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")


Starting model training on CPU... (This will be slow 🐢)
--- Epoch 1/15 ---


Validation: 100%|██████████| 20/20 [02:04<00:00,  6.22s/it]


Train Loss: 0.7934, Train Acc: 0.7603
Val Loss: 0.6027, Val Acc: 0.8670

New best model saved with accuracy: 0.8670
--- Epoch 2/15 ---


Validation: 100%|██████████| 20/20 [02:02<00:00,  6.11s/it]


Train Loss: 0.6010, Train Acc: 0.8669
Val Loss: 0.5538, Val Acc: 0.8942

New best model saved with accuracy: 0.8942
--- Epoch 3/15 ---


...

Validation: 100%|██████████| 20/20 [02:00<00:00,  6.01s/it]


Train Loss: 0.3589, Train Acc: 0.9956
Val Loss: 0.5234, Val Acc: 0.9215

--- Epoch 14/15 ---


Validation: 100%|██████████| 20/20 [02:07<00:00,  6.39s/it]


Train Loss: 0.3566, Train Acc: 0.9976
Val Loss: 0.5276, Val Acc: 0.9199

--- Epoch 15/15 ---


Training:  77%|███████▋  | 60/78 [20:36<06:08, 20.46s/it, loss=0.352]